In [3]:
from pathlib import Path
import os
from openai import OpenAI
import logging
from datetime import date
import pandas as pd
import numpy as np
import json


PROJ_DIR = Path("/Users/ashikawaharuki/Desktop/research/TDB/test/99_system")
DATA_DIR = PROJ_DIR/"data"
TABLES_DIR = DATA_DIR/"tables"
PROMPT_DIR = DATA_DIR/"prompt"

In [9]:
array = np.array(["nj", "aa", "nj", None])
np.unique(array).size


2

In [8]:
df = pd.read_csv(TABLES_DIR/"amazon-purchases.csv")
df

def filter_df_by_parents(panrents, df):
    condition = True
    for k, v in panrents.items():
        condition &= (df[k]==v)
    return df[condition]

df_ = filter_df_by_parents(panrents={"Shipping Address State":"MA"}, df=df)
df_

,Order Date,Purchase Price Per Unit,Quantity,Shipping Address State,Title,ASIN/ISBN (Product Code),Category,Survey ResponseID
4352,2020-08-10,8.98,1.0,MA,"Oh, the Places You'll Go!",0679805273,ABIS_BOOK,R_0BrMxYYMHVMOBEJ
4353,2020-08-10,4.68,1.0,MA,Giraffes Can't Dance,0545392551,ABIS_BOOK,R_0BrMxYYMHVMOBEJ
4354,2020-08-10,5.67,1.0,MA,"Brown Bear, Brown Bear, What Do You See?",0805047905,ABIS_BOOK,R_0BrMxYYMHVMOBEJ
4355,2020-08-10,27.95,1.0,MA,Safe and Sound Crib Mattress Protector (2 Pack...,B07PJY9Q3C,MATTRESS_COVER,R_0BrMxYYMHVMOBEJ
4356,2020-12-13,8.99,1.0,MA,"Sassy Rain Shower Bath Ball STEM Bath Toy, 6+ ...",B07VQ92YLB,BABY_PRODUCT,R_0BrMxYYMHVMOBEJ
...,...,...,...,...,...,...,...,...
1850711,2021-04-01,20.75,1.0,MA,50 Feather (Yellow Label) Razor Blades Platinu...,B08P5K9GMJ,RAZOR_BLADE_CARTRIDGE,R_zfqnsBzlOAKibzb
1850712,2021-04-01,6.99,1.0,MA,Tanner's Tasty Paste Vanilla Bling - Anticavit...,B015ZRTHVA,TOOTH_CLEANING_AGENT,R_zfqnsBzlOAKibzb
1850713,2021-04-14,15.99,1.0,MA,Sinland Microfiber Cleaning Cloth Dish Cloth K...,B00QGCXPRG,TOWEL,R_zfqnsBzlOAKibzb
1850714,2021-05-22,6.99,4.0,MA,Tanner's Tasty Paste Vanilla Bling - Anticavit...,B015ZRTHVA,TOOTH_CLEANING_AGENT,R_zfqnsBzlOAKibzb


In [3]:
"""
入力: (att_l, att_label_d)=(["売上","従業員数","上場flg"],{"売上":"Quantitative","上場flg":"Categorical",...})
出力: {"売上":[1,3,5],"従業員数":[1,3,5],"上場flg":[2]}
"""
def make_operation(att_l, att_label_d):
    result_d = {}
    operation_d = {1:"合計値を計算(sum)",2:"平均値を計算(mean)",3:"最大値を計算(max)",4:"最小値を計算(min)",5:"中央値を計算(meadian)",6:"データ数をカウント(count)",7:"ユニークな値の数をカウント(nunique)",8:"ユニークな値を列挙(unique)"}
    for att_name in att_l:
        instruction = f"「{att_name}」属性に適応したい操作を以下から選択し、その数字をカンマ(,)区切りで入力してください"
        if(att_label_d[att_name]=="Quantitative"):
            choices = "\n".join([f"{k}:{v}" for k, v in operation_d.items() if(k<=6)])
            user_choice = [int(n) for n in input(instruction+"\n\n"+choices+"\n").split(",")]
        else:
            choices = "\n".join([f"{k}:{v}" for k, v in operation_d.items() if(k>=6)])
            user_choice = [int(n) for n in input(instruction+"\n\n"+choices+"\n").split(",")]
        result_d[att_name] = user_choice
    return result_d
make_operation(att_l=["売上","従業員数","上場flg"],att_label_d={"売上":"Quantitative","従業員数":"Quantitative","上場flg":"Categorical"})

{'売上': [1, 2, 6], '従業員数': [1, 2, 6], '上場flg': [7]}

In [10]:
def make_subject(parents, drilldown_attribute, df, flg=True):
    if(flg):
        df = filter_df_by_parents(panrents=parents,df=df)
    return [[parents, drilldown_attribute, value] for value in df[drilldown_attribute].unique()]

make_subject(parents={"Shipping Address State":"MA"}, drilldown_attribute="Category", df=df)

[[{'Shipping Address State': 'MA'}, 'Shipping Address State', 'MA']]

In [11]:
df = pd.read_csv(DATA_DIR/"amazon-purchases.csv")
array = df["Shipping Address State"].nunique(dropna=False)
np.unique(array).size


1